In [1]:
import pandas as pd 
import os
from tqdm import tqdm
tqdm.pandas()
from datetime import datetime
import re, html, copy
import seaborn as sns

In [26]:
tweet_filepath = os.path.join('C:\DATA\ComBERT\data_finetuning_spamFiltering', 'tweets_imbalanced_286399.csv')
company_filepath = os.path.join('C:\DATA\ComBERT\data', 'company_info_sec_cik_mapper_12057_20220802.csv')

save_test_filepath_format = os.path.join('C:\DATA\ComBERT\data_finetuning_spamFiltering', 'test_{}_{}.csv')
save_train_filepath_format = os.path.join('C:\DATA\ComBERT\data_finetuning_spamFiltering', 'train_{}.csv')

Dataset for classification task

In [5]:
df = pd.read_csv(tweet_filepath)
df.columns = ['raw_text', *df.columns[1:]]
df.head()

,raw_text,is_retweet,label,created_at,yyyymm
0,RT @curtmelonopoly: $AUPH If price stays in th...,True,human,2017-05-18 22:00:41,201705
1,$WFM $SFM $UNFI$NGVC Natural GrocersI like it ...,False,human,2017-05-18 22:01:00,201705
2,Check out $ATHN on ChartMill at https://t.co/N...,False,human,2017-05-18 22:01:25,201705
3,Check out $AR on ChartMill at https://t.co/zlX...,False,human,2017-05-18 22:01:25,201705
4,Check out $KMX on ChartMill at https://t.co/DX...,False,human,2017-05-18 22:01:25,201705


## Preprocessing
* RT @Account_Name: -> [RT]
* URL -> [URL] 
* @Account_Name -> ' '
* Ticker -> Company name (ex. $TSLA -> Tesla Inc)

In [7]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"
def preprocess_clean(sent):
    sent = sent.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sent = re.sub("\s+", " ", sent)
    sent = sent.lower()
    return sent

def preprocess_trim(sent):
    sent = re.sub("\s+", " ", sent)
    sent = sent.strip()
    return sent

rt_pattern = re.compile('RT @[a-zA-Z]*:')
url_pattern = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
account_pattern = re.compile('@[a-zA-Z1-9]*')
multiple_spaces_pattern = re.compile(' +', re.UNICODE)
def preprocess(text):
    text = re.sub(rt_pattern, '[RT]', text)
    text = re.sub(url_pattern, '[URL]', text)
    text = re.sub(account_pattern, ' ', text)
    text = re.sub(multiple_spaces_pattern, ' ', text)
    
    text = preprocess_clean(text)
    text = convert_ticker_with_cashtag_to_name(text)
    text = preprocess_trim(text)
    
    return text

company_df = pd.read_csv(company_filepath)
company_df.dropna(subset=['Ticker', 'Name'], inplace=True)
company_df['Ticker'] = company_df['Ticker'].apply(lambda x: x.lower())
company_df['Name'] = company_df['Name'].apply(lambda x: x.lower())

convert_ticker_to_name = dict(zip('$'+company_df.Ticker, company_df.Name))
def convert_ticker_with_cashtag_to_name(text):
    return re.sub(r'\$([a-zA-Z.-]+)', lambda m: convert_ticker_to_name.setdefault(m.group(0), m.group(0)), text.lower())

company_df

,CIK,Ticker,Name,Exchange
0,1750,air,aar corp,NYSE
1,1800,abt,abbott laboratories,NYSE
2,1961,wddd,worlds inc,OTC
3,2098,acu,acme united corp,NYSE
4,2178,ae,"adams resources & energy, inc.",NYSE
...,...,...,...,...
12052,1931691,mobv,mobiv acquisition corp,NaN
12053,1933644,mdlvy,"medlive technology co., ltd./adr",NaN
12054,1934064,osa,laaa merger corp.,NaN
12055,1934945,tbmcu,trailblazer merger corp i,NaN


In [8]:
df['text'] = df['raw_text'].progress_apply(lambda x: preprocess(x))
df.head()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 286399/286399 [00:37<00:00, 7707.30it/s]


,raw_text,is_retweet,label,created_at,yyyymm,text
0,RT @curtmelonopoly: $AUPH If price stays in th...,True,human,2017-05-18 22:00:41,201705,[rt] aurinia pharmaceuticals inc. if price sta...
1,$WFM $SFM $UNFI$NGVC Natural GrocersI like it ...,False,human,2017-05-18 22:01:00,201705,"$wfm sprouts farmers market, inc. united natur..."
2,Check out $ATHN on ChartMill at https://t.co/N...,False,human,2017-05-18 22:01:25,201705,check out $athn on chartmill at [url] #digital...
3,Check out $AR on ChartMill at https://t.co/zlX...,False,human,2017-05-18 22:01:25,201705,check out antero resources corp on chartmill a...
4,Check out $KMX on ChartMill at https://t.co/DX...,False,human,2017-05-18 22:01:25,201705,check out carmax inc on chartmill at [url] #di...


Preprocessing Example

In [9]:
row = df.iloc[-1]
print('{}\n=> {}\n'.format(row['raw_text'], row['text']))

row = df.iloc[-2]
print('{}\n=> {}'.format(row['raw_text'], row['text']))

$AAPL Can do this for a while w/o need for reinventing the wheel. They got the system of interconnected products/services @IsaacCheatham88
=> apple inc. can do this for a while w o need for reinventing the wheel they got the system of interconnected products services

RT @MindMakesMatter: $NVDA daily updatedDamn thing hasn't even blown yet!Recheck those last 2 quarters. if you wanna https://t.co/rIKVq…
=> [rt] nvidia corp daily updateddamn thing hasn t even blown yet recheck those last 2 quarters if you wanna [url]


## Constructing balanced train/test set

In [10]:
print('(201709) Before splitting, maximum number of rows for one class:', len(df[(df['label']=='bot')&(df['yyyymm']==201709)]))
print('(201709) Before splitting, maximum number of rows for one class:', len(df[(df['label']=='human')&(df['yyyymm']==201709)]))
print('(!201709) Before splitting, maximum number of rows for one class:', len(df[(df['label']=='bot')&(df['yyyymm']!=201709)]))
print('(!201709) Before splitting, maximum number of rows for one class:', len(df[(df['label']=='human')&(df['yyyymm']!=201709)]))

(201709) Before splitting, maximum number of rows for one class: 6992
(201709) Before splitting, maximum number of rows for one class: 18107
(!201709) Before splitting, maximum number of rows for one class: 41150
(!201709) Before splitting, maximum number of rows for one class: 220150


### Test datasets
* Produced in September 2017
* human:bot = 1:1

In [14]:
test_df = pd.concat([df[(df['yyyymm']==201709)&(df['label']==label)] for label in df['label'].unique()])
test_df

,raw_text,is_retweet,label,created_at,yyyymm,text
261300,1897 $ZOES Sep $12.5 PUTS 95% BID (IV=52.6% -4...,False,human,2017-09-01 00:01:00,201709,1897 $zoes sep $12 5 puts 95 bid iv 52 6 4 9 o...
261301,$LTC IS LIT! #WDYM CAN'T TWEET THIS ENOUGH SEE...,False,human,2017-09-01 00:01:15,201709,ltc properties inc is lit #wdym can t tweet th...
261302,@CockroachCapLP Indeed. Yet I hear some of the...,False,human,2017-09-01 00:02:17,201709,indeed yet i hear some of these are small ish ...
261304,"Good technicals, but also a nice setup pattern...",False,human,2017-09-01 00:04:04,201709,good technicals but also a nice setup pattern ...
261305,ECA Could Offer Some Great Trade Op\u2026 $ECA...,False,human,2017-09-01 00:05:05,201709,eca could offer some great trade op u2026 $eca...
...,...,...,...,...,...,...
286378,RT @LegendsRoom: The #Vegas #crypto takeover c...,True,bot,2017-09-12 21:47:50,201709,[rt] the #vegas #crypto takeover continues $lg...
286384,RT @TTOON_STOCKS: #REVERSE #Merger Rocket Phar...,True,bot,2017-09-12 21:48:57,201709,rt _stocks: #reverse #merger rocket pharmaceut...
286385,RT @TTOON_STOCKS: $ITEK AH #Merger #News Inote...,True,bot,2017-09-12 21:49:04,201709,rt _stocks: $itek ah #merger #news inotek phar...
286395,RT @LauraJKeller: Missed flurry of bank news f...,True,bot,2017-09-12 21:57:29,201709,[rt] missed flurry of bank news from conf catc...


In [16]:
def contains_company_name(text, company_name_list):
    for company_name in company_name_list:
        if company_name in text:
            return True
    return False
test_df['contains_company_name'] = test_df['text'].progress_apply(lambda x: contains_company_name(x, company_df['Name'].values))
test_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25099/25099 [00:42<00:00, 589.53it/s]


,raw_text,is_retweet,label,created_at,yyyymm,text,contains_company_name
261300,1897 $ZOES Sep $12.5 PUTS 95% BID (IV=52.6% -4...,False,human,2017-09-01 00:01:00,201709,1897 $zoes sep $12 5 puts 95 bid iv 52 6 4 9 o...,False
261301,$LTC IS LIT! #WDYM CAN'T TWEET THIS ENOUGH SEE...,False,human,2017-09-01 00:01:15,201709,ltc properties inc is lit #wdym can t tweet th...,True
261302,@CockroachCapLP Indeed. Yet I hear some of the...,False,human,2017-09-01 00:02:17,201709,indeed yet i hear some of these are small ish ...,True
261304,"Good technicals, but also a nice setup pattern...",False,human,2017-09-01 00:04:04,201709,good technicals but also a nice setup pattern ...,True
261305,ECA Could Offer Some Great Trade Op\u2026 $ECA...,False,human,2017-09-01 00:05:05,201709,eca could offer some great trade op u2026 $eca...,False
...,...,...,...,...,...,...,...
286378,RT @LegendsRoom: The #Vegas #crypto takeover c...,True,bot,2017-09-12 21:47:50,201709,[rt] the #vegas #crypto takeover continues $lg...,False
286384,RT @TTOON_STOCKS: #REVERSE #Merger Rocket Phar...,True,bot,2017-09-12 21:48:57,201709,rt _stocks: #reverse #merger rocket pharmaceut...,False
286385,RT @TTOON_STOCKS: $ITEK AH #Merger #News Inote...,True,bot,2017-09-12 21:49:04,201709,rt _stocks: $itek ah #merger #news inotek phar...,False
286395,RT @LauraJKeller: Missed flurry of bank news f...,True,bot,2017-09-12 21:57:29,201709,[rt] missed flurry of bank news from conf catc...,True


In [27]:
test_df_containing_company_name = pd.concat([test_df[(test_df['contains_company_name']==True)&(test_df['label']==label)].sample(500) for label in test_df['label'].unique()])

save_filepath = save_test_filepath_format.format('containing_company_name', len(test_df_containing_company_name))
test_df_containing_company_name.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

test_df_without_company_name = pd.concat([test_df[(test_df['contains_company_name']==False)&(test_df['label']==label)].sample(500) for label in test_df['label'].unique()])

save_filepath = save_test_filepath_format.format('without_company_name', len(test_df_without_company_name))
test_df_without_company_name.to_csv(save_filepath, index=False)
print('Created {}'.format(save_filepath))

Created C:\DATA\ComBERT\data_finetuning_spamFiltering\test_containing_company_name_1000.csv
Created C:\DATA\ComBERT\data_finetuning_spamFiltering\test_without_company_name_1000.csv


### Train dataset
* Produced in May, June, July, and August 2017

In [42]:
train_full_df = df[df['yyyymm']!=201709]

filepath = save_train_filepath_format.format(len(train_full_df))
train_full_df.to_csv(filepath, index=False)
print('Created {}'.format(filepath))

Created C:\DATA\ComBERT\data_finetuning\train_261300.csv
